# Analyze Dust Bayesian Inference Data

In [ ]:
using Plots,Random,Measures,CSV,DataFrames,Statistics
pyplot();

include("aux.jl"); include("poimcmc.jl");

In [ ]:
prm,_=data();
println("Measured copies/mg dust:")
println(prm[:Y])

## Load the MCMC samples

In [ ]:
df = CSV.read("GibbsMCMC.csv",DataFrame,header=false); sort!(df,:Column1); nsmp = size(df)[2]-1;
df[!,1] = Symbol.(df[!,1]);
println("Number of MCMC samples: $nsmp");

## Posterior distributions for key parameters
#### Parse parameters

In [ ]:
PRM = Vector{Dict{Symbol,Float64}}(undef,nsmp);
shift = 1;

for i=1:nsmp
    # load parameters
    prm,_ = rdprm(df[:,i+1],df[:,1]);
    PRM[i] = deepcopy(prm);
end 

#### Number infected

In [ ]:
n = Vector{Float64}(undef,nsmp)
for i=1:nsmp
    n[i] = Int64(floor(PRM[i][:n]));
end

In [ ]:
p1 = histogram(n,normalize=:pdf,labels="",title="posterior",xlabel="number infected",ylabel="density");
p2 = plot(n,linewidth=1,labels="",title="trace",xlabel="smp",ylabel="number infected");

plot(p1,p2,size=(600,300),margin=2mm)

#### Individual shedding curves
$\Gamma(\alpha,\beta)$ distribution hyperparameters for shedding amplitude

In [ ]:
Γₛ = Array{Float64,2}(undef,2,nsmp);
for i=1:nsmp
        Γₛ[1,i]=PRM[i][:Γα];
        Γₛ[2,i]=PRM[i][:Γβ];
end

In [ ]:
# Joint distribution of Γ(α,β)-hyperparameters
println("Posterior joint distribution:")
histogram2d(Γₛ[1,:],Γₛ[2,:],xlabel="Γα",ylabel="Γβ",title="Shedding Distribution")

$\Gamma(\bar{\alpha},\bar{\beta})$ distribution for posterior mean hyperparams

In [ ]:
function myΓ(α::Float64,β::Float64,x::Float64)
    val = exp(α*log(β))*exp((α-1)*log(x))*exp(-β*x)/gamma(α);
    
    return val
end

function myΓ(α::Float64,β::Float64,x::Vector{Float64})
    n = length(x);
    val = Vector{Float64}(undef,n);
    for i=1:n
        val[i] = myΓ(α,β,x[i]);
    end
    
    return val
end;

In [ ]:
mα,mβ = sum(Γₛ,dims=2)/nsmp;
μ = mα/mβ; var = μ/mβ;
prmrg,_ = mcmcrg();

Ax = convert(Vector,LinRange(prmrg[:A1][1],prmrg[:A1][2],1000));
Ay = myΓ(mα,mβ,Ax);

println("Mean Γ-distribution for hyperparameters:")
println("(mα,mβ)=($mα,$mβ)")
println("(mean,var)=($μ,$var)")
plot(Ax,Ay,linewidth=3,title="Γ-distr",labels="",size=(300,250))

$A$ amplitude of shedding

In [ ]:
A = Array{Float64,1}(undef,0);
for i=1:nsmp
    niso = PRM[i][:niso] |> floor |> Int64;
    n = PRM[i][:n] |> floor |> Int64;
    ram = Vector{Float64}(undef,n)
    
    for j=(niso+1):(niso+n)
        Anow = Symbol(:A,j);
        ram[j-niso] = PRM[i][Anow];
    end
    
    A = [A;ram]
end

In [ ]:
p1 = histogram(A,normalize=:pdf,labels="",title="posterior",xlabel="Amplitude peak",ylabel="density",xlims=(0,quantile(A,0.95)));
p2 = plot(A,linewidth=1,labels="",title="trace",xlabel="smp",ylabel="Amplitude peak");

plot(p1,p2,size=(600,300),margin=2mm)

$A_x$ position of peak

In [ ]:
Aₓ = Array{Float64,1}(undef,0);
for i=1:nsmp
    niso = PRM[i][:niso] |> floor |> Int64;
    n = PRM[i][:n] |> floor |> Int64;
    ram = Vector{Float64}(undef,n)
    
    for j=(niso+1):(niso+n)
        Anow = Symbol(:Aₓ,j);
        ram[j-niso] = PRM[i][Anow];
    end
    
    Aₓ = [Aₓ;ram]
end

In [ ]:
p1 = histogram(Aₓ,normalize=:pdf,labels="",title="posterior",xlabel="position of peak",ylabel="density");
p2 = plot(Aₓ,linewidth=1,labels="",title="trace",xlabel="smp",ylabel="position of peak");

plot(p1,p2,size=(600,300),margin=2mm)

#### Poisson mean for total surviving virus shed 

In [ ]:
Pμ = Vector{Float64}(undef,nsmp);
nmax = Int64(floor(PRM[1][:nmax])); λval = Vector{Float64}(undef,nmax);
for i=1:nsmp
    niso = Int64(floor(PRM[i][:niso])); n = Int64(floor(PRM[i][:n]));
    shedλ!(PRM[i];λval=λval);
    val = 0.0;
    for j=(niso+1):(niso+n)
        val += λval[j]
    end
    Pμ[i] = val;
end

In [ ]:
p1 = histogram(Pμ,normalize=:pdf,labels="",title="posterior",xlabel="mean virus detected",ylabel="density",
               xlims=(0.00,quantile(Pμ,0.95)));
p2 = plot(Pμ,linewidth=1,labels="",title="trace",xlabel="smp",ylabel="mean virus detected");

plot(p1,p2,size=(600,300),margin=2mm)

## MCMC diagnostics
#### Rejection statistics

In [ ]:
dfrej = CSV.read("RejStats.csv",DataFrame);
mhrate = sum(dfrej[!,:mhrej])/sum(dfrej[!,:mhtot]);

println("The random walk Metropolis-Hastings rejection rate: $mhrate");